<font style="font-size:96px; font-weight:bolder; color:#0040a0"><img src="http://montage.ipac.caltech.edu/docs/M51_logo.png" alt="M" style="float: left; padding: 25px 30px 25px 0px;" /></font>

<i><b>Montage</b> Montage is an astronomical image toolkit with components for reprojection, background matching, coaddition and visualization of FITS files. It can be used as a set of command-line tools (Linux, OS X and Windows), C library calls (Linux and OS X) and as Python binary extension modules.

The Montage source is written in ANSI-C and code can be downloaded from GitHub ( https://github.com/Caltech-IPAC/Montage ). The Python package can be installed from PyPI ("</i>pip install MontagePy<i>"). The package has no external dependencies. See http://montage.ipac.caltech.edu/ for details on the design and applications of Montage.


# MontagePy.main modules: mShrinkCube

Most of Montage is focused on 2D astronomical image mosaics: reprojection, background matching, coaddition and so on.  But there is also a need for tools that operate on data "cubes": three- or four-dimensional arrays where two of the axes represent the same projected sky coordinates as we find in the 2D images.  Usually, the third axis is some form of wavelength.

The problem sets are not completely parallel: datacubes do not generally need background matching but you do frequently want cutouts in the none spatial dimensions and to transpose axes.

Montage includes a set of routines for manipulating datacubes:

<ul>
<li><b>mmProjectCube</b> &mdash; Reproject the spatial dimensions.</li>
<li><b>mAddCube</b> &mdash; Reproject the cube.</li>
<li><b>mShrinkCube</b> &mdash; Rescale a cube (integer scaling in the non-spatial dimensions).</li>
<li><b>mSubCube</b> &mdash;Cut a portion out of a cube.</li>
<li><b>mTranspose</b> &mdash; Transpose a cube's axes.</li>
</ul>

This routine, mShrinkCube, resizes a cube.  In the spatial dimensions it works just like mShrink and supports floating point rescale factors.  In the third (and fourth) dimensions it only supports integer resampling (<i>e.g.</i>, averaging N input wavelengths for each output wavelength).

<b>Note:</b> The MontagePy python package has no external dependencies. We include other utilities on this page to aid in visualizing MontagePy package results.

In [1]:
from MontagePy.main import mShrinkCube, mViewer

help(mShrinkCube)

Help on built-in function mShrinkCube in module MontagePy.main:

mShrinkCube(...)
    mShrinkCube is a utility program for making smaller versions of a FITS file by averaging NxN blocks of pixels spatially and M values in the third and fourth cube dimensions.  N can be fractional but M must be an integer.
    
    Parameters
    ----------
    infile : str
        Input FITS file.
    output_file : str
        Subimage output FITS file.
    shrinkFactor : float
        Scale factor for spatial shrinking.  Can be any positive real number.
    mfactor : int
        Positive integer scale factor for shrinking the third cube dimension.
    hdu : int, optional
        Optional HDU offset for input file.
    fixedSize : bool, optional
        Alternate mode: shrink so the output fits in this many pixels.
    debug : int, optional
        Debugging output level.
    
    
    Returns
    -------
    status : int
        Return status (0: OK, 1:ERROR).
    msg : str
        Return message (for

## mShrinkCube Example

In the spatial dimensions, mShrinkCube behaves much the same as mShrink, shrinking or expanding an image by a non-integer factor.  In the third cube dimension this factor is an optional parameter and must be an integer.

The data used in the datacube Jupyter pages come from the Galactic Arecibo L-band Feed Array HI (GALFA-HI) survey (Peek et al., 2011, Ap J Suppl, 194, 20; DOI 10.1088/0067-0049/194/2/20; ADS Bibcode 2011ApJS..194...20P).  These are fairly large files so what we do here is to shrink it down by a factor of ten by averaging wavelengths.

In [2]:
rtn = mShrinkCube('GALFA/shrunken/GALFA_HI_RA+DEC_004.00+34.35_N.fits', 
                  'work/GALFA/GALFA_HI_RA+DEC_004.00+34.35_N_m2_2.fits', 
                  2, 2)
print(rtn)

{'status': '0', 'time': 1.0}


## Before and After

Here are the original image and the resampled one. Since these are cubes, we have to collapse it in the third dimension for display. To show the effects of the resampling, we will show 1035 through 1045 of the input and 99 through 109 of the output.  These ranges are the same width (10) and centered on the same original wavelength (104)


In [6]:
from IPython.display import HTML, display

rtn = mViewer('-gray "GALFA/shrunken/GALFA_HI_RA+DEC_004.00+34.35_N.fits[0][60,68]" \
               -2s max gaussian-log \
               -out work/GALFA/GALFA_HI_RA+DEC_004.00+34.35_N_shrinkcube.png',
              '', mode=2)

rtn = mViewer('-gray "work/GALFA/GALFA_HI_RA+DEC_004.00+34.35_N_m2_2.fits[0][30,32]" \
               -2s max gaussian-log \
               -out work/GALFA/GALFA_HI_RA+DEC_004.00+34.35_N_m2_2_shrinkcube.png',
              '', mode=2)

display(HTML("<table><tr><td><img src='work/GALFA/GALFA_HI_RA+DEC_004.00+34.35_N_shrinkcube.png'></td> \
                         <td><img src='work/GALFA/GALFA_HI_RA+DEC_004.00+34.35_N_m2_2_shrinkcube.png'></td></tr></table>"))

,


&nbsp;<p/>

## mShrinkCube Error Handling

If mShrinkCube encounters an error, the return structure will just have two elements: a status of 1 ("error") and a message string that tries to diagnose the reason for the error.

For instance, if the user specifies a cube that doesn't exist: 

In [7]:
rtn = mShrinkCube('GALFA/shrunken/unknown.fits', 
                  'work/GALFA/GALFA_HI_RA+DEC_004.00+34.35_N_m2_2.fits', 
                  2, 2)
print(rtn)

{'status': '1', 'msg': b'Image file GALFA/shrunken/unknown.fits missing or invalid FITS'}


&nbsp;

## Classic Montage: mShrinkCube as a Stand-Alone Program

### mShrinkCube Unix/Windows Command-line Arguments

<p>mShrinkCube can also be run as a command-line tool in Linux, OS X, and Windows:</p>

<p><tt>
<b>Usage:</b> mShrinkCube [-f(ixed-size)] [-d level] [-h hdu] [-m factor] [-s statusfile] in.fits out.fits factor
</tt></p>
<p>&nbsp;</p>
<p>If you are writing in C/C++, mShrinkCube can be accessed as a library function:</p>


<pre>
/*-***********************************************************************/
/*                                                                       */
/*  mCubeShrink                                                          */
/*                                                                       */
/*  This module, is a utility program for making smaller versions of a   */
/*  FITS file by averaging NxN blocks of pixels spatially and M values   */
/*  in the third and fourth cube dimensions.  N can be fractional but    */
/*  M must be an integer.                                                */
/*                                                                       */
/*   char  *infile         Input FITS file                               */
/*   char  *output_file    Shrunken output FITS file                     */
/*                                                                       */
/*   double shrinkFactor   Scale factor for spatial shrinking.  Can be   */
/*                         any positive real number                      */
/*                                                                       */
/*   int    mfactor        Positive integer scale factor for shrinking   */
/*                         the third cube dimension                      */
/*                                                                       */
/*   int    hdu            Optional HDU offset for input file            */
/*                                                                       */
/*   int    fixedSize      Alternate mode: shrink so the output fits     */
/*                         in this many pixels                           */
/*                                                                       */
/*   int    debug          Debugging output level                        */
/*                                                                       */
/*************************************************************************/

struct mShrinkCubeReturn *mShrinkCube(char *input_file, char *output_file, double shrinkFactor, 
                                int mfactor, int hduin, int fixedSize, int debug)
</pre>
<p><b>Return Structure</b></p>
<pre>
struct mShrinkCubeReturn
{
   int    status;        // Return status (0: OK, 1:ERROR)
   char   msg [1024];    // Return message (for error return)
   char   json[4096];    // Return parameters as JSON string
   double time;          // Run time (sec)   
};
</pre>